In [66]:

import PyPDF2
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [67]:
file_path = r'C:\Users\dell\Downloads\DataSet_FineTunning.pdf' 
with open(file_path, 'rb') as pdf_file:
    reader = PyPDF2.PdfReader(pdf_file)
    text = ""
    # Extract text from each page
    for page in reader.pages[2:]:
        text += page.extract_text()

In [68]:
def clean_texte(text):
    cleaned_text=re.sub(r'\s+', ' ', text)
    cleaned_text = re.sub(r'[^\x00-\x7F]+', '', cleaned_text)
    return cleaned_text.strip()

In [69]:
clean_texte=clean_texte(text)

In [70]:
print(clean_texte)

3 Loi n 01-00 portant organisation de l'enseignement suprieur Principes et objectifs Article premier L'enseignement suprieur, objet de la prsente loi, est fond sur les principes suivants : Il est dispens dans le cadre du respect des principes et vale urs de la foi islamique qui prsident  son dveloppement et  so n volution. Il est ouvert  tous les citoyens remplissant les cond itions requises sur la base de l'galit des chances. Il est exerc selon les principes des droits de l'Homme, de tol rance, de libert de pense, de cration et d'innovation, dans le strict respect des rgles et des valeurs acadmiques d'objectiv it, de rigueur scientifique et d'honntet intellectuelle. Il relve de la responsabilit de l'Etat qui en assure la pl anification, l'organisation, le dveloppement, la rgulation et l'orientation selon les besoins conomiques, sociaux et culturels de la Nation, qui en dfinit la politique nationale avec le concours de la communaut scientifique, du monde du travail et de l'conomie ains

In [71]:
def chunk_fixed_length(text, chunk_size):
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]


fixed_length_chunks = chunk_fixed_length(clean_texte,700)

In [72]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained model from Hugging Face
model = SentenceTransformer('all-MiniLM-L6-v2')
# Convert text chunks to vectors
vectors = model.encode(fixed_length_chunks)

# Print the shape of the vectors (should be [number of chunks, vector size])
print(vectors.shape)

(93, 384)


In [73]:
!pip install faiss-cpu
import faiss
import numpy as np

# Define the dimension of your vectors (in this case, it is 384 for 'all-MiniLM-L6-v2')
vector_dim = 384

# Create a FAISS index (IndexFlatL2 is a basic index for L2 distance)
index = faiss.IndexFlatL2(vector_dim)

# Add your vectors to the index
index.add(np.array(vectors))

# Check how many vectors are indexed
print(f"Number of vectors indexed: {index.ntotal}")

Number of vectors indexed: 93


In [74]:
# Encode the user query into a vector
query = "Quelles sont les missions principales des universités ?"
query_vector = model.encode([query])

# Search for the top 5 most similar chunks in the FAISS index
top_k = 5
distances, indices = index.search(np.array(query_vector), top_k)

# Print the top results (text chunks)
for idx in indices[0]:
    print(f"Relevant section: {fixed_length_chunks[idx]}")

relevant_chunks = [fixed_length_chunks[idx] for idx in indices[0][:5]]
context = " ".join(relevant_chunks)
prompt = f"Hi , ansewr the user my question based on the following context \n\nContext: {context} The users question is this : {query}"


Relevant section:  L'enseignement suprieur public est assur dans les universits ainsi que dans les tablissements d'enseignement suprieur ne relevant pas des universits. Les enseignements sont dispenss dans les facults, les coles d'ingnieurs prcdes de classes prparatoires, les coles et instituts suprieurs, les institutions de formation de cadres pdagogiques et de fo rmation de techniciens spcialiss ou quivalents. L'enseignement suprieur public peut galement tre assur dans des cycles spcifiques de prparation aux mtiers organiss, soit au sein des universits, soit au sein d'institutions suprieures existantes ou spcialement cres  cet effet. Chapitre premier Des universits Article 3 Les universits ont pour missions
Relevant section: ics. Article 5 Dans le cadre de l'exercice des missions qui leur sont im parties, les universits jouissent de l'autonomie pdagogique, scientifique et culturelle, sous rserv e des dispositions de la prsente loi. Certaines activits de formation et de r echerche peu

In [75]:
import os
from dotenv import load_dotenv

# Load environment variables
result = load_dotenv(r'C:\Users\dell\Data Science\RAG\config.env.txt')

# Access the API key
my_api_key = os.getenv('GOOGLE_API_KEY')

if my_api_key:
    print("API key loaded successfully:")
else:
    print("Failed to load API key. Check your config file.")


API key loaded successfully:


In [76]:
# Install the Google Generative AI package if needed
!pip install google-generativeai

import google.generativeai as genai
import os


genai.configure(api_key=my_api_key)

context = context
# Configure the model
model_name = 'gemini-1.5-flash'  # Choose the appropriate gemini model
temperature = 1.0  # Adjust as necessary for creativity control

# Set up model and configuration
model = genai.GenerativeModel(model_name)
config = genai.GenerationConfig(temperature=temperature)

# Generate response using the prompt
response = model.generate_content(contents=[prompt], generation_config=config)

# Print the generated summary
print(response.text)


D'après le texte, les missions des universités ne sont pas explicitement listées dans l'article 3, qui mentionne seulement qu'elles ont "pour missions ics."  Cependant, l'article 5 indique que les universités jouissent d'une autonomie pédagogique, scientifique et culturelle dans l'exercice de leurs missions.  On peut déduire que ces missions englobent au minimum la formation et la recherche, vu que certaines activités dans ces domaines peuvent faire l'objet de contrats pluriannuels avec l'État.  L'article 6 précise également qu'elles sont pluridisciplinaires et peuvent être spécialisées, et regroupent des établissements d'enseignement, de formation et de recherche.

